VERSION NOTES:
- Fama-MacBeth Function & Statistical Significance tests

# Asset Pricing Tests

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels.panel.model import FamaMacBeth
from linearmodels.asset_pricing import TradedFactorModel
from linearmodels.asset_pricing import LinearFactorModelGMM as GMM

## Data

In [2]:
ff_data = pd.read_csv('ff_data.csv', index_col=0, parse_dates=True)
monthly_factors = pd.read_csv('monthly_factors.csv', index_col=0, parse_dates=True)
quarterly_factors = pd.read_csv('quarterly_factors.csv', index_col=0, parse_dates=True)

In [3]:
# Monthly
# Test asset clean up
ff_data = ff_data.loc[monthly_factors.index[0]:monthly_factors.index[-1]] / 100

# Factor Models
ff3 = ff_data.iloc[:,0:3]
ff5 = ff_data.iloc[:,0:5]
ff5_mom = pd.concat([ff5, ff_data.iloc[:,31]], axis=1)

# Test Asset Portfolios
size_book_assets_m = ff_data.iloc[:,6:31].apply(lambda x: x - ff_data['RF'])
industry_assets_m = ff_data.iloc[:,32:].apply(lambda x: x - ff_data['RF'])

# Macro Factors
factors_m = pd.concat([monthly_factors, ff_data['Mkt-RF']], axis=1)

In [4]:
# Quarterly
def f(data, last_row=False):
    '''
    Computes the buy and hold for 3 month (1 quarter) return.
    '''
    df = ((1+data).cumprod(axis=0)-1)

    return df.iloc[-1]

ff_data_q = (ff_data).resample('Q').apply(f).iloc[1:-2]

# Factor Models
ff3_q = ff_data_q.iloc[:,0:3]
ff5_q = ff_data_q.iloc[:,0:5]
ff5_mom_q = pd.concat([ff5_q, ff_data_q.iloc[:,31]], axis=1)

# Test Asset Portfolios
size_book_assets_q = ff_data_q.iloc[:,6:31].apply(lambda x: x - ff_data_q['RF'])
industry_assets_q = ff_data_q.iloc[:,32:].apply(lambda x: x - ff_data_q['RF'])

# Macro Factors
factors_q = pd.concat([quarterly_factors, ff_data_q['Mkt-RF']], axis=1).dropna()

## Analysis

Two sets of test assets will be used: 
1) 25 Size-Book Sorted Portfolios 
2) 30 Industry Portfolios

Several factor models will be tested (on each set of test assets):
1) Macro Factors
    - how well do they price the cross-section?
    - what are their risk premia if any?
2) Macro Factors + Market Factor
3) FF3
    - Compare Macro Factors to FF3
4) FF3 + Macro Factors
    - Given the Macro Factors are SMB + HML needed?
5) FF5
6) FF5 + Momentum
    - do the Macro Factors price better than the standard model?
7) FF5 + Momentum + Macro Factors
    - is the standard model subsumed by the Macro Factors?

### Linear Factor Model/SDF Approach

In [61]:
# Renames the factors to make subsequent code cleaner
# _0 denotes MoM or QoQ, _1 denotes YoY or 4QoQ
factors_m.columns = factors_m.columns.str.replace("mom", "0")
factors_m.columns = factors_m.columns.str.replace("yoy", "1")

factors_q.columns = factors_q.columns.str.replace("qoq", "0")
factors_q.columns = factors_q.columns.str.replace("4qoq", "1")

In [69]:
def factor_models(test_assets, factors):
    '''
    Runs all factor models on the given set of test assets.
    Stores the model instances in a dictionary.
    '''
    
    model_results = {}
    
    # Macro Factors
    model1 = TradedFactorModel(test_assets, factors.iloc[:,:2]).fit(cov_type='kernel')
    model_results['macro'] = model1
   
    # Macro Factors + Market
    model2 = TradedFactorModel(test_assets, factors.iloc[:,:]).fit(cov_type='kernel')
    model_results['macro+mkt'] = model2
   
    # FF3
    model3 = TradedFactorModel(test_assets, ff3).fit(cov_type='kernel')
    model_results['ff3'] = model3
    
    # FF3 + Macro Factors
    model4 = TradedFactorModel(test_assets, pd.concat([ff3, factors.iloc[:,:2]], axis=1)).fit(cov_type='kernel')
    model_results['ff3+macro'] = model4
    
    # FF5 + Momentum
    model5 = TradedFactorModel(test_assets, ff5_mom).fit(cov_type='kernel')
    model_results['ff5+mom'] = model5
     
    # FF5 + Momentum + Macro Factors
    model6 = TradedFactorModel(test_assets, pd.concat([ff5_mom, factors.iloc[:,:2]], axis=1)).fit(cov_type='kernel')
    model_results['ff5+macro'] = model6
    
    return model_results

#### Monthly

##### Test Assets: 25 Size-Book Value Sorted Portfolios

In [72]:
size_book_models_mom = factor_models(size_book_assets_m, factors_m[['gdp_0', 'cpi_0', 'Mkt-RF']])
size_book_models_yoy = factor_models(size_book_assets_m, factors_m[['gdp_1', 'cpi_1', 'Mkt-RF']])

In [76]:
# dir(size_book_models_mom['macro+mkt'])
# size_book_models_mom['macro+mkt'].full_summary
# size_book_models_mom['macro+mkt'].betas
# size_book_models_mom['macro+mkt'].alphas

###### MoM Model

In [74]:
size_book_models_mom['macro+mkt']

TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 25   R-squared:                      0.7276
No. Factors:                          3   J-statistic:                    89.046
No. Observations:                   595   P-value                         0.0000
Date:                  Sat, Mar 18 2023   Distribution:                 chi2(25)
Time:                          17:53:21                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
gdp_0          0.0002     0.0001     1.4004     0.1614  -6.241e-05      0.0004
cpi_0         -0.0001  3.603e-05    -2.9049     0.0037     -0.0002  -3.405e-05
Mkt-RF         0.0061     0.0020     3.1136     0.0018      0.0023      0.0099
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 12
See full_summary for complete results
LinearFactorModelResults, id: 0x2ad5f50c6d0

GDP's risk premia becomes insignificant (not a surprise from the factor creation stage).

CPI's premia is highly significant, negative and very small.

In [75]:
size_book_models_mom['macro+mkt']._jstat

J-statistic
H0: All alphas are 0
Statistic: 89.0458
P-value: 0.0000
Distributed: chi2(25)
WaldTestStatistic, id: 0x2ad5f4aed00

In [77]:
12*size_book_models_mom['macro+mkt'].alphas.sort_values()

SMALL LoBM   -0.070035
ME2 BM1      -0.026450
ME3 BM1      -0.022094
BIG LoBM     -0.002565
ME4 BM1       0.000389
ME5 BM4       0.002473
ME5 BM2       0.011611
ME1 BM2       0.012548
ME4 BM2       0.012643
ME1 BM3       0.015558
ME5 BM3       0.015651
ME2 BM2       0.015721
BIG HiBM      0.017111
ME4 BM3       0.018343
ME3 BM3       0.019770
ME3 BM2       0.023827
ME2 BM3       0.024028
ME4 BM4       0.024414
ME3 BM4       0.032213
ME4 BM5       0.033756
ME2 BM4       0.036667
ME2 BM5       0.039072
ME1 BM4       0.044953
ME3 BM5       0.049590
SMALL HiBM    0.053801
Name: alpha, dtype: float64

Reject null that alphas are 0 for the Macro + Mkt Model; the model does not price the portfolios.

The J-statistic itself is the lowest amoung all the models (even FF5+Momentum+Macro). The biggest pricing errors are in the more extreme portfolios (particularly the "SMALL" portfolios).

In [80]:
size_book_models_mom['ff5+mom']._jstat

J-statistic
H0: All alphas are 0
Statistic: 91.4080
P-value: 0.0000
Distributed: chi2(25)
WaldTestStatistic, id: 0x2ad5f556880

In [82]:
size_book_models_mom['ff5+macro']._jstat

J-statistic
H0: All alphas are 0
Statistic: 80.5827
P-value: 0.0000
Distributed: chi2(25)
WaldTestStatistic, id: 0x2ad5f57d340

Adding macro to FF5 + Momentum enhances pricing (marginally).

In [83]:
size_book_models_mom['ff3+macro']

TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 25   R-squared:                      0.9174
No. Factors:                          5   J-statistic:                    83.636
No. Observations:                   595   P-value                         0.0000
Date:                  Sat, Mar 18 2023   Distribution:                 chi2(25)
Time:                          17:53:21                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Mkt-RF         0.0061     0.0019     3.1229     0.0018      0.0023      0.0099
SMB            0.0022     0.0012     1.7837     0.0745     -0.0002      0.0046
HML            0.0030     0.0016     1.9047     0.0568  -8.807e-05      0.0062
gdp_0          0.0002     0.0001     1.3668     0.1717  -6.779e-05      0.0004
cpi_0         -0.0001  3.598e-05    -2.9089     0.0036     -0.0002  -3.415e-05
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 9
See full_summary for complete results
LinearFactorModelResults, id: 0x2ad5f5569d0

Macro factors do not subsume SMB and HML; GDP remains insignificant while CPI is still highly significant.

###### YoY Model

In [93]:
size_book_models_yoy['macro+mkt']

TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 25   R-squared:                      0.7263
No. Factors:                          3   J-statistic:                    87.233
No. Observations:                   595   P-value                         0.0000
Date:                  Sat, Mar 18 2023   Distribution:                 chi2(25)
Time:                          17:53:21                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
gdp_1          0.0004     0.0002     1.9826     0.0474   4.684e-06      0.0008
cpi_1         -0.0005     0.0002    -2.4412     0.0146     -0.0009  -9.523e-05
Mkt-RF         0.0061     0.0019     3.1261     0.0018      0.0023      0.0099
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 7
See full_summary for complete results
LinearFactorModelResults, id: 0x2ad5f5fb760

In [94]:
size_book_models_yoy['macro+mkt']

TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 25   R-squared:                      0.7263
No. Factors:                          3   J-statistic:                    87.233
No. Observations:                   595   P-value                         0.0000
Date:                  Sat, Mar 18 2023   Distribution:                 chi2(25)
Time:                          17:53:21                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
gdp_1          0.0004     0.0002     1.9826     0.0474   4.684e-06      0.0008
cpi_1         -0.0005     0.0002    -2.4412     0.0146     -0.0009  -9.523e-05
Mkt-RF         0.0061     0.0019     3.1261     0.0018      0.0023      0.0099
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 7
See full_summary for complete results
LinearFactorModelResults, id: 0x2ad5f5fb760

In [100]:
size_book_models_yoy['ff3+macro']

TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 25   R-squared:                      0.9175
No. Factors:                          5   J-statistic:                    82.995
No. Observations:                   595   P-value                         0.0000
Date:                  Sat, Mar 18 2023   Distribution:                 chi2(25)
Time:                          17:53:21                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Mkt-RF         0.0061     0.0019     3.1184     0.0018      0.0023      0.0099
SMB            0.0022     0.0012     1.7794     0.0752     -0.0002      0.0046
HML            0.0030     0.0016     1.8917     0.0585     -0.0001      0.0062
gdp_1          0.0004     0.0002     2.0255     0.0428   1.326e-05      0.0008
cpi_1         -0.0005     0.0002    -2.3973     0.0165     -0.0009  -8.813e-05
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 10
See full_summary for complete results
LinearFactorModelResults, id: 0x2ad5f64ba60

#### Quarterly

##### Test Assets: 25 Size-Book Value Sorted Portfolios

In [72]:
size_book_models_mom = factor_models(size_book_assets_m, factors_m[['gdp_0', 'cpi_0', 'Mkt-RF']])
size_book_models_yoy = factor_models(size_book_assets_m, factors_m[['gdp_1', 'cpi_1', 'Mkt-RF']])

In [76]:
# dir(size_book_models_mom['macro+mkt'])
# size_book_models_mom['macro+mkt'].full_summary
# size_book_models_mom['macro+mkt'].betas
# size_book_models_mom['macro+mkt'].alphas

###### QoQ Model

In [74]:
size_book_models_mom['macro+mkt']

TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 25   R-squared:                      0.7276
No. Factors:                          3   J-statistic:                    89.046
No. Observations:                   595   P-value                         0.0000
Date:                  Sat, Mar 18 2023   Distribution:                 chi2(25)
Time:                          17:53:21                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
gdp_0          0.0002     0.0001     1.4004     0.1614  -6.241e-05      0.0004
cpi_0         -0.0001  3.603e-05    -2.9049     0.0037     -0.0002  -3.405e-05
Mkt-RF         0.0061     0.0020     3.1136     0.0018      0.0023      0.0099
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 12
See full_summary for complete results
LinearFactorModelResults, id: 0x2ad5f50c6d0

GDP's risk premia becomes insignificant (not a surprise from the factor creation stage).

CPI's premia is highly significant, negative and very small.

In [75]:
size_book_models_mom['macro+mkt']._jstat

J-statistic
H0: All alphas are 0
Statistic: 89.0458
P-value: 0.0000
Distributed: chi2(25)
WaldTestStatistic, id: 0x2ad5f4aed00

In [77]:
12*size_book_models_mom['macro+mkt'].alphas.sort_values()

SMALL LoBM   -0.070035
ME2 BM1      -0.026450
ME3 BM1      -0.022094
BIG LoBM     -0.002565
ME4 BM1       0.000389
ME5 BM4       0.002473
ME5 BM2       0.011611
ME1 BM2       0.012548
ME4 BM2       0.012643
ME1 BM3       0.015558
ME5 BM3       0.015651
ME2 BM2       0.015721
BIG HiBM      0.017111
ME4 BM3       0.018343
ME3 BM3       0.019770
ME3 BM2       0.023827
ME2 BM3       0.024028
ME4 BM4       0.024414
ME3 BM4       0.032213
ME4 BM5       0.033756
ME2 BM4       0.036667
ME2 BM5       0.039072
ME1 BM4       0.044953
ME3 BM5       0.049590
SMALL HiBM    0.053801
Name: alpha, dtype: float64

Reject null that alphas are 0 for the Macro + Mkt Model; the model does not price the portfolios.

The J-statistic itself is the lowest amoung all the models (even FF5+Momentum+Macro). The biggest pricing errors are in the more extreme portfolios (particularly the "SMALL" portfolios).

In [80]:
size_book_models_mom['ff5+mom']._jstat

J-statistic
H0: All alphas are 0
Statistic: 91.4080
P-value: 0.0000
Distributed: chi2(25)
WaldTestStatistic, id: 0x2ad5f556880

In [82]:
size_book_models_mom['ff5+macro']._jstat

J-statistic
H0: All alphas are 0
Statistic: 80.5827
P-value: 0.0000
Distributed: chi2(25)
WaldTestStatistic, id: 0x2ad5f57d340

Adding macro to FF5 + Momentum enhances pricing (marginally).

In [83]:
size_book_models_mom['ff3+macro']

TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 25   R-squared:                      0.9174
No. Factors:                          5   J-statistic:                    83.636
No. Observations:                   595   P-value                         0.0000
Date:                  Sat, Mar 18 2023   Distribution:                 chi2(25)
Time:                          17:53:21                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Mkt-RF         0.0061     0.0019     3.1229     0.0018      0.0023      0.0099
SMB            0.0022     0.0012     1.7837     0.0745     -0.0002      0.0046
HML            0.0030     0.0016     1.9047     0.0568  -8.807e-05      0.0062
gdp_0          0.0002     0.0001     1.3668     0.1717  -6.779e-05      0.0004
cpi_0         -0.0001  3.598e-05    -2.9089     0.0036     -0.0002  -3.415e-05
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 9
See full_summary for complete results
LinearFactorModelResults, id: 0x2ad5f5569d0

Macro factors do not subsume SMB and HML; GDP remains insignificant while CPI is still highly significant.

###### 4QoQ Model

##### Test Assets: 30 Industry Portfolios

In [90]:
industry_models_mom = factor_models(industry_assets_m, factors_m[['gdp_0', 'cpi_0', 'Mkt-RF']])
industry__models_yoy = factor_models(industry_assets_m, factors_m[['gdp_1', 'cpi_1', 'Mkt-RF']])

In [91]:
industry_models_mom['macro+mkt']

TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 30   R-squared:                      0.5589
No. Factors:                          3   J-statistic:                    70.963
No. Observations:                   595   P-value                         0.0000
Date:                  Sat, Mar 18 2023   Distribution:                 chi2(30)
Time:                          18:02:57                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
gdp_0          0.0002     0.0001     1.3788     0.1680  -6.584e-05      0.0004
cpi_0         -0.0001  3.584e-05    -2.9205     0.0035     -0.0002  -3.443e-05
Mkt-RF         0.0061     0.0019     3.1184     0.0018      0.0023      0.0099
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 10
See full_summary for complete results
LinearFactorModelResults, id: 0x2ad5e7b5580

### Fama-MacBeth

In [74]:
def fama_macbeth(test_assets, factors):
    '''
    Function to perform Fama-MacBeth procedure.
    '''
    from scipy.stats import chi2
    
    # Time Series Regression
    betas = []
    
    for portfolio in test_assets:
        ts = sm.OLS(test_assets.loc[:,portfolio], sm.add_constant(factors)).fit()
        betas.append(ts.params[1:])

    betas = pd.DataFrame(betas)
    betas.index = test_assets.columns
    
    # Cross Sectional Regressions
    T = len(factors)
    lambdas = []
    alphas = []
    
    for t in range(T):
        cs = sm.OLS(np.array(test_assets)[t].T, betas).fit()
        lambdas.append(cs.params)
        alphas.append(cs.resid)
            
    # Risk Premia significance (without controlling for autocorrelation)
    lambdas_df = pd.DataFrame(lambdas)
    
    std_errors = lambdas_df.apply(lambda col: np.sqrt(((col - col.mean())**2).sum() / T**2))
    t_stats = lambdas_df.mean() / std_errors
    
    premia = pd.merge(lambdas_df.mean().rename('Risk Premia'), t_stats.rename('t-stat'), left_index=True, right_index=True)
    
    # Alpha significance (without controlling for autocorrelation)
    alphas_df = pd.DataFrame(alphas)
    alpha_hat = alphas_df.mean()

    cov_alpha = ((alphas_df - alpha_hat).T @ (alphas_df - alpha_hat)) / len(alphas_df)**2
    
    alpha_chi = alpha_hat.T @ np.linalg.inv(cov_alpha) @ alpha_hat
    alpha_p_val = 1 - chi2.cdf(alpha_chi, df=len(alpha_hat)-len(premia))

    alpha_hat_mean = abs(alpha_hat).mean()
    alpha = pd.DataFrame({'Test Statistic': alpha_chi, 'p-value':alpha_p_val, 'Mean Absolute alpha':alpha_hat_mean}, index=['H0: alphas=0'])
    
    return (premia, alpha)

In [75]:
def multiple_models(start='1973', end='2022'):
    '''
    Function to run several pricing models.
    Start and end allow for custom dates.
    '''
    
    # Models
    macro_model = fama_macbeth(size_book_assets_q.loc[start:end], factors_q[['gdp_4qoq', 'cpi_4qoq', 'Mkt-RF']].loc[start:end])
    ff3 = fama_macbeth(size_book_assets_q.loc[start:end], ff3_q.loc[start:end])
    ff5_mom = fama_macbeth(size_book_assets_q.loc[start:end], ff5_mom_q.loc[start:end])
    
    # Output
    premia = pd.concat({'Macro Model':macro_model[0],'FF3':ff3[0], 'FF5 + MoM':ff5_mom[0]}) 
    premia.index.names = ["Model", "Factors"]
    
    alpha = pd.concat({'Macro Model':macro_model[1],'FF3':ff3[1], 'FF5 + MoM':ff5_mom[1]}, names=['Model', 'Hypothesis'])
    alpha.index = alpha.index.swaplevel(0,1)
    
    return (premia, alpha)

In [76]:
# Full-Sample (1973Q3 - 2022Q2)
full_sample0 = multiple_models()[0]
full_sample1 = multiple_models()[1]

display(full_sample0)
display(full_sample1)

Risk Premia    t-stat
Model       Factors                        
Macro Model gdp_4qoq     0.003206  1.548907
            cpi_4qoq    -0.008500 -2.610649
            Mkt-RF       0.020838  3.137645
FF3         Mkt-RF       0.018907  2.958602
            SMB          0.007564  1.898064
            HML          0.011000  2.347356
FF5 + MoM   Mkt-RF       0.019605  3.077580
            SMB          0.009498  2.410665
            HML          0.010194  2.192431
            RMW          0.012871  2.297801
            CMA          0.000906  0.192922
            Mom          0.030342  2.494422

Test Statistic   p-value  Mean Absolute alpha
Hypothesis   Model                                                     
H0: alphas=0 Macro Model       59.828639  0.000024             0.002736
             FF3              204.452267  0.000000             0.002918
             FF5 + MoM       -284.239043  1.000000             0.002057

In [77]:
# Vassalou Sample ( - 1998Q4)
pre0 = multiple_models(end='1998')[0]
pre1 = multiple_models(end='1998')[1]

display(pre0)
display(pre1)

Risk Premia    t-stat
Model       Factors                        
Macro Model gdp_4qoq     0.010423  3.976937
            cpi_4qoq    -0.009193 -3.229644
            Mkt-RF       0.022742  2.593065
FF3         Mkt-RF       0.021559  2.481653
            SMB          0.003977  0.697368
            HML          0.014559  2.578230
FF5 + MoM   Mkt-RF       0.021865  2.526267
            SMB          0.005455  0.959350
            HML          0.013430  2.387979
            RMW          0.011122  2.117136
            CMA         -0.005922 -0.957865
            Mom         -0.017195 -1.589295

Test Statistic   p-value  Mean Absolute alpha
Hypothesis   Model                                                     
H0: alphas=0 Macro Model      173.407904  0.000000             0.003773
             FF3               55.487784  0.000101             0.002219
             FF5 + MoM         49.232997  0.000170             0.001786

In [78]:
# Post-Vassalou Sample (1998Q4 - )
post0 = multiple_models(start='1999')[0]
post1 = multiple_models(start='1999')[1]

display(post0)
display(post1)

Risk Premia    t-stat
Model       Factors                        
Macro Model gdp_4qoq     0.003332  1.110859
            cpi_4qoq     0.001314  0.466648
            Mkt-RF       0.019345  2.032859
FF3         Mkt-RF       0.017113  1.822992
            SMB          0.009338  1.724411
            HML          0.007344  0.966730
FF5 + MoM   Mkt-RF       0.020385  2.183998
            SMB          0.008693  1.641231
            HML          0.006981  0.931696
            RMW          0.007695  0.911447
            CMA          0.006165  0.933939
            Mom          0.041546  2.862085

Test Statistic       p-value  Mean Absolute alpha
Hypothesis   Model                                                         
H0: alphas=0 Macro Model       85.313759  2.121909e-09             0.004348
             FF3               99.475143  7.965739e-12             0.004339
             FF5 + MoM         58.203384  7.419005e-06             0.002610

In [32]:
from texttable import Texttable
import latextable

In [67]:
rows = [
        [" ", "1973Q3-2022Q4", "Pre 1998Q4", "Post 1998Q4"],
        ["GDP Factor", premia[0], premia[1], premia[2]],
        [" ", tstat[0], tstat[1], tstat[2]],
        
        ["CPI Factor", premia[3], premia[4], premia[5]],
        [" ", tstat[3], tstat[4], tstat[5]],
    
        ["GDP Factor", premia[6], premia[7], premia[8]],
        [" ", tstat[6], tstat[7], tstat[8]],
       ]
multicolumn_header = [("Factors", 1), ("Full-Sample", 1), ("Pre-Publication", 1), ("Post-Publication", 1)]
print('\n-- Example 11: Multicolumn header with drop column --')
print('Latextable Output:')
print(latextable.draw_latex(rows, use_booktabs=True, multicolumn_header=multicolumn_header))


-- Example 11: Multicolumn header with drop column --
Latextable Output:
\begin{table}
	\begin{center}
		\begin{tabular}{llll}
			\toprule
			\multicolumn{1}{c}{Factors} & \multicolumn{1}{c}{Full-Sample} & \multicolumn{1}{c}{Pre-Publication} & \multicolumn{1}{c}{Post-Publication} \\
			  & 1973Q3-2022Q4 & Pre 1998Q4 & Post 1998Q4 \\
			\midrule
			GDP Factor & 0.003 & 0.010 & 0.003 \\
			  & 1.549 & 3.977 & 1.111 \\
			CPI Factor & -0.009 & -0.009 & 0.001 \\
			  & -2.611 & -3.230 & 0.467 \\
			GDP Factor & 0.021 & 0.023 & 0.019 \\
			  & 3.138 & 2.593 & 2.033 \\
			\bottomrule
		\end{tabular}
	\end{center}
	\caption{t-statistics in brackets}
\end{table}


#### Full-Sample (testing code)

In [56]:
beta = []
for portfolio in size_book_assets_q:
    ts = sm.OLS(size_book_assets_q.loc[:,portfolio], sm.add_constant(factors_q[['gdp_4qoq', 'cpi_4qoq', 'Mkt-RF']])).fit()
    beta.append(ts.params[1:])

beta = pd.DataFrame(beta)
beta.index = size_book_assets_q.columns

In [57]:
lambdas = []
alphas = []
for t in range(0,size_book_assets_q.index.shape[0]):
    cs = sm.OLS(np.array(size_book_assets_q)[t].T, beta).fit()
    lambdas.append(cs.params)
    alphas.append(cs.resid)

In [58]:
lambdas_df = pd.DataFrame(lambdas)
lambdas_df.mean()

gdp_4qoq    0.003206
cpi_4qoq   -0.008500
Mkt-RF      0.020838
dtype: float64

In [62]:
se_gdp = np.sqrt(((lambdas_df['gdp_4qoq'] - lambdas_df.mean()['gdp_4qoq'])**2).sum() / len(lambdas_df)**2)
se_cpi = np.sqrt(((lambdas_df['cpi_4qoq'] - lambdas_df.mean()['cpi_4qoq'])**2).sum() / len(lambdas_df)**2)

In [63]:
# Fama-MacBeth t-stat without controlling for autocorrelation 
print(f'{lambdas_df.mean()["gdp_4qoq"]/se_gdp:0.5f}', f'{lambdas_df.mean()["cpi_4qoq"]/se_cpi:0.5f}')

1.54891 -2.61065


In [96]:
# Joint test of alphas
alphas_df = pd.DataFrame(alphas)
alpha_hat = alphas_df.mean()

In [122]:
cov_alpha = ((alphas_df - alpha_hat).T @ (alphas_df - alpha_hat)) / len(alphas_df)**2

In [124]:
# Chi Squared Stat
alpha_chi = alpha_hat.T @ np.linalg.inv(cov_alpha) @ alpha_hat
alpha_chi

59.82863873966363

In [125]:
from scipy.stats import chi2

In [126]:
alpha_p_val = 1 - chi2(len(alphas_df)-1).cdf(alpha_chi)
alpha_p_val

1.0

#### 5 year Rolling Window

##### 1st Stage

In [30]:
from statsmodels.regression.rolling import RollingOLS

In [31]:
beta = {}
for portfolio in size_book_assets:
    ts = RollingOLS(size_book_assets.loc[size_book_assets.index,portfolio],factors.iloc[:,0:3], window=60).fit()
    beta[portfolio] = ts.params[1:]

In [32]:
betas_T = {}
for t in range(60,len(size_book_assets)-60):
    betas = []
    for portfolio in size_book_assets.columns:
        betas.append(beta[portfolio].iloc[t])
    
    betas_df = pd.DataFrame(np.array(betas), index=size_book_assets.columns, columns=['const', 'gdp_factor', 'cpi_factor'])
    
    betas_T[t] = betas_df

##### 2nd Stage

In [33]:
lambdas = []
alphas = []
for t in range(60,len(size_book_assets)-60):
    cs = sm.OLS(np.array(size_book_assets)[t].T, betas_T[t]).fit()
    lambdas.append(cs.params)
    alphas.append(cs.resid)

In [34]:
pd.DataFrame(lambdas).mean()

const         0.901521
gdp_factor    0.000045
cpi_factor   -0.001267
dtype: float64

In [401]:
# pd.DataFrame(alphas).mean()

### GMM?

In [108]:
gmm1 = GMM(size_book_assets_m, factors_m[['gdp_1', 'cpi_1', 'Mkt-RF']])
gmm2 = GMM(size_book_assets_m, pd.concat([ff3, factors_m.iloc[:,2:4]], axis=1))

In [109]:
gmm1_res = gmm1.fit(cov_type='kernel')

Iteration: 0, Objective: 284.35765759144397
Iteration: 10, Objective: 166.61463063480804
Iteration: 20, Objective: 112.43348892070674
Iteration: 30, Objective: 97.67247415456143
Iteration: 40, Objective: 86.1489110180111
Iteration: 50, Objective: 75.52432964734759
Iteration: 60, Objective: 58.25409123682409
Iteration: 70, Objective: 46.065607160290355
Iteration: 80, Objective: 36.95367315436503
Iteration: 90, Objective: 29.34127159769115
Iteration: 100, Objective: 28.105193696573053
Iteration: 110, Objective: 28.095730997216172
Iteration: 120, Objective: 28.095708105701718
         Current function value: 28.095708
         Iterations: 125
         Function evaluations: 12640
         Gradient evaluations: 154
Iteration: 0, Objective: 22.879137516286413
Iteration: 10, Objective: 21.98751332196565
Iteration: 20, Objective: 21.425959592263006
Iteration: 30, Objective: 21.229546893065297
Iteration: 40, Objective: 20.870641138567258
Iteration: 50, Objective: 20.714669095145013
Iteration: 6

In [110]:
gmm1_res

LinearFactorModelGMM Estimation Summary                     
================================================================================
No. Test Portfolios:                 25   R-squared:                      0.7255
No. Factors:                          3   J-statistic:                    19.939
No. Observations:                   595   P-value                         0.5868
Date:                  Sat, Mar 18 2023   Distribution:                 chi2(22)
Time:                          19:01:14                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
gdp_1          0.0116     0.0030     3.9318     0.0001      0.0058      0.0174
cpi_1         -0.0054     0.0018    -2.9483     0.0032     -0.0090     -0.0018
Mkt-RF         0.0061     0.0016     3.8491     0.0001      0.0030      0.0093
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 8
See full_summary for complete results
GMMFactorModelResults, id: 0x2ad5e940970

In [111]:
gmm2_res = gmm2.fit(cov_type='kernel')

Iteration: 0, Objective: 693.6529504331223
Iteration: 10, Objective: 466.8414783269898
Iteration: 20, Objective: 277.11025800633297
Iteration: 30, Objective: 209.83413277996218
Iteration: 40, Objective: 183.55818588498303
Iteration: 50, Objective: 168.51240381640116
Iteration: 60, Objective: 155.00248253722515
Iteration: 70, Objective: 148.39463774636258
Iteration: 80, Objective: 145.0649766000757
Iteration: 90, Objective: 141.22128544327597
Iteration: 100, Objective: 111.90739013390687
Iteration: 110, Objective: 75.40205730941659
Iteration: 120, Objective: 67.65021920629324
Iteration: 130, Objective: 59.31214131131045
Iteration: 140, Objective: 47.728145522170216
Iteration: 150, Objective: 32.64430781953283
Iteration: 160, Objective: 29.769942953597855
Iteration: 170, Objective: 27.562268459597835
Iteration: 180, Objective: 26.912825746027607
Iteration: 190, Objective: 26.671395920207274
Iteration: 200, Objective: 26.591219803849853
Iteration: 210, Objective: 26.553450687310356
Iterat

In [112]:
gmm2_res

LinearFactorModelGMM Estimation Summary                     
================================================================================
No. Test Portfolios:                 25   R-squared:                      0.9145
No. Factors:                          5   J-statistic:                    18.769
No. Observations:                   595   P-value                         0.5369
Date:                  Sat, Mar 18 2023   Distribution:                 chi2(20)
Time:                          19:03:12                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Mkt-RF         0.0061     0.0016     3.8155     0.0001      0.0029      0.0092
SMB            0.0013     0.0011     1.1939     0.2325     -0.0008      0.0034
HML            0.0036     0.0013     2.8745     0.0040      0.0012      0.0061
gdp_1          0.0101     0.0027     3.7010     0.0002      0.0047      0.0154
cpi_1         -0.0048     0.0018    -2.5769     0.0100     -0.0084     -0.0011
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 8
See full_summary for complete results
GMMFactorModelResults, id: 0x2ad5e940880

In [114]:
gmm3 = GMM(size_book_assets_m, ff3)
gmm3.fit()

Iteration: 0, Objective: 136.76574712681216
Iteration: 10, Objective: 103.70303814648729
Iteration: 20, Objective: 99.50976657515605
Iteration: 30, Objective: 98.79269412370823
Iteration: 40, Objective: 98.50661820482655
Iteration: 50, Objective: 97.71832133250547
Iteration: 60, Objective: 93.07160047668103
Iteration: 70, Objective: 92.92568933040663
Iteration: 80, Objective: 91.39316784072614
         Current function value: 90.692977
         Iterations: 89
         Function evaluations: 11327
         Gradient evaluations: 138
Iteration: 0, Objective: 89.59615228730786
Iteration: 10, Objective: 88.79220643896886
Iteration: 20, Objective: 88.63714983349956
Iteration: 30, Objective: 88.62489954591607
Iteration: 40, Objective: 88.61645832820784
Iteration: 50, Objective: 88.59603848337271
Iteration: 60, Objective: 88.46878876342907
Iteration: 70, Objective: 88.46336095689942
Iteration: 80, Objective: 88.39442744892398
Iteration: 90, Objective: 88.37439370951921
         Current function

LinearFactorModelGMM Estimation Summary                     
================================================================================
No. Test Portfolios:                 25   R-squared:                      0.9163
No. Factors:                          3   J-statistic:                    88.374
No. Observations:                   595   P-value                         0.0000
Date:                  Sat, Mar 18 2023   Distribution:                 chi2(22)
Time:                          19:04:29                                         
Cov. Estimator:                  robust                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Mkt-RF         0.0083     0.0019     4.4517     0.0000      0.0047      0.0120
SMB            0.0021     0.0012     1.6989     0.0893     -0.0003      0.0044
HML            0.0042     0.0012     3.3979     0.0007      0.0018      0.0066
==============================================================================

Covariance estimator:
HeteroskedasticCovariance
See full_summary for complete results
GMMFactorModelResults, id: 0x2ad5e94eb50